# Dimensions in CNNs

Understanding dimensions (or how to calculate the number of parameters) will help you make accurate tradeoffs between model size and performance. As you'll see, some parameters have a much bigger impact on model size than others.

In general, for a single data point, transformations of data from one layer are of the form $g(w \cdot x + b)$, where $g$ is an activation function, $w$ is a vector of weights, $\cdot$⋅ is a dot product (sum of an elementwise product), and $b$ is an offset or bias term (a single number in this example). The model learns the weights $w$ and offset $b$ through a numerical optimization scheme. Convolutional layers work similarly, but the data is arranged a little differently.

<img src="images/a_layer_example.png" alt="Drawing"/>

A single image fed into a convolutional layer $l$ is represented as a 3-dimensional array, where each element represents a pixel intensity. This image has pixel height $n_H^{[l-1]}$, pixel width $n_W^{[l-1]}$, and the number of channels (aka depth) $n_C^{[l-1]}$. 
* In the first convolutional layer, the images fed in are original images that are either RGB images or grayscale images. The number of channels $n_C^{[0]}$ is either 3 or 1. 
* In a intermediate convolutional layer $l$, the images fed in are transformed images (also called feature maps) outputed from previous layer. The number of channels $n_C^{[l-1]}$ can be arbitrary number depeneding on the network architect.

A single filter is also represented as a 3-dimensional array. It has a height and width $f^{[1]}$ (height and width are typically the same) and channels $n_C^{[l-1]}$. 
> Note that the number of channels of the filter must match the number of channels of the image upon which the filter acts. This is why we denote the channels of the filter as $n_C^{[l-1]}$, the output channels of previous convolutional layer.

A convolutional layer can have multiple filters each is to detect a different type of features. Each filter produces a channel of the output image. Therefore, the number of filters defined in a convolutional layer equals the number of channels of the output images of that convolution layer. We denote the output channels for layer $l$ as $n_C^{[l]}$.

By now, we know that a filter in layer $l$ has shape $(f^{[1]}, f^{[1]}, n_C^{[l-1]})$ and there are $n_C^{[l]}$ of them. Therefore, the shape of all filters (you can treat them as a big filter) is $(f^{[1]}, f^{[1]}, n_C^{[l-1]}, n_C^{[l]})$.

Corresponding to each output channel, there is a bias/offset $b$. Therefore, for $n_C^{[l]}$ number of output channels, there are $n_C^{[l]}$ number of bias. The total number of parameters should be $f^{[1]} \times f^{[1]} \times n_C^{[l-1]} \times n_C^{[l]} + n_C^{[l]}$.

### To summary

For a convolutional layer $l$, we have:
* $f^{l}$ is filter size
* $p^{l}$ is padding
* $s^{l}$ is stride
* $n_C^{l}$ is number of filters
* shape of an input image: $(n_H^{l-1}, n_W^{l-1}, n_C^{l-1})$
* shape of an output image: $(n_H^{l}, n_W^{l}, n_C^{l})$
* The formulars to calculate $n_H^{l}$ and $n_W^{l}$ are:
$$n_H^{l} = \lfloor \frac{n_H^{l-1} + 2p^{l} - f^{l}}{s^{l}} + 1 \rfloor $$
$$n_W^{l} = \lfloor \frac{n_W^{l-1} + 2p^{l} - f^{l}}{s^{l}} + 1 \rfloor $$

Then, 

* Each filter has shape of $(f^{[1]}, f^{[1]}, n_C^{[l-1]})$. There are $n_C^{l}$ of them. All filters have the shape of $(f^{[1]}, f^{[1]}, n_C^{[l-1]}, n_C^{[l]})$. The number of weights for all the filters are $f^{[1]} \times f^{[1]} \times n_C^{[l-1]} \times n_C^{[l]}$.
* There are $n_C^{[l]}$ number of bias, and the corresponding offset array of bias is $(1,1,1,n_C^{[l]})$
* The totoal number of parameters $f^{[1]} \times f^{[1]} \times n_C^{[l-1]} \times n_C^{[l]} + n_C^{[l]}$
* The shape of output image (or activation) is $(n_H^{l}, n_W^{l}, n_C^{l})$. For a minibatch of $m$ examples, the output shape should be $(m, n_H^{l}, n_W^{l}, n_C^{l})$

**ConvetNet Example**

<img src="images/convnet_example.png" alt="Drawing"/>



**Quiz**

* H = height, W = width, D = depth
* We have an input of shape 32x32x3 (HxWxD)
* 20 filters of shape 8x8x3 (HxWxD)
* A stride of 2 for both the height and width (S)
* With padding of size 1 (P)
* Recall the formula for calculating the new height or width:

> <b style='color:blue'>new_height</b> = <b style='color:red'>(input_height - filter_height + 2 * P)/S + 1</b>

> <b style='color:blue'>new_width</b> = <b style='color:red'>(input_width - filter_width + 2 * P)/S + 1</b>

* What's the shape of the output? 

We can get the new height and width with the formula resulting in:

> (32 - 8 + 2 * 1)/2 + 1 = 14 

> (32 - 8 + 2 * 1)/2 + 1 = 14

The new depth is equal to the number of filters, which is 20.
This would correspond to the following code:

```python
input = tf.placeholder(tf.float32, (None, 32, 32, 3))

# filter has the dimension: (height, width, input_depth, output_depth)
filter_weights = tf.Variable(tf.truncated_normal((8, 8, 3, 20))) 

filter_bias = tf.Variable(tf.zeros(20))

# stride has the dimension: (batch, height, width, depth)
strides = [1, 2, 2, 1] 
padding = 'SAME'
conv = tf.nn.conv2d(input, filter_weights, strides, padding) + filter_bias
```

Note the output shape of conv will be [1, 16, 16, 20]. It's 4D to account for batch size, but more importantly, it's not [1, 14, 14, 20]. 

> This is because the padding algorithm TensorFlow uses is not exactly the same as the one above. 

An alternative algorithm is to switch padding from 'SAME' to 'VALID' which would result in an output shape of [1, 13, 13, 20]. If you're curious how padding works in TensorFlow, read this [document](https://www.tensorflow.org/api_guides/python/nn#Convolution).

The filter is applied to image patches of the same size as the filter and strided according to the strides argument, which has the dimension: (batch, height, width, depth). 

> strides = [1, 1, 1, 1] applies the filter to a patch at every offset.

> strides = [1, 2, 2, 1] applies the filter to every other image patch in each dimension.

In summary TensorFlow uses the following equation for 'SAME' vs 'VALID'

<b>SAME Padding</b>, the output height and width are computed as:

> <b style='color:blue'>out_height</b> = <b style='color:red'>ceil( float(in_height) / float(strides[1]) )</b>

> <b style='color:blue'>out_width</b> = <b style='color:red'>ceil( float(in_width) / float(strides[2]) )</b>

<b>VALID Padding</b>, the output height and width are computed as:

> <b style='color:blue'>out_height</b> = <b style='color:red'>ceil( float(in_height - filter_height + 1 ) / float(strides[1]))</b>

> <b style='color:blue'>out_width</b> = <b style='color:red'>ceil( float(in_width - filter_width + 1 ) / float(strides[2]))</b>